In [ ]:
# файл candidates_posts.csv, в данных не сохранился, но результат работы данного ноутбука
# сохранен в файле candidates_posts_labelled в jkh_groups_data.tar.gz
!gdown 114jOepwvcfAxa--w9SoJjucaz4TeAHpD

!pip install openai backoff geopy

Downloading...
From: https://drive.google.com/uc?id=114jOepwvcfAxa--w9SoJjucaz4TeAHpD
To: /content/candidates_posts.csv
100% 5.39M/5.39M [00:00<00:00, 249MB/s]
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.6/73.6 kB 2.5 MB/s eta 0:00:00


In [ ]:
import os
import json
import concurrent.futures

import openai
from openai.error import RateLimitError
import backoff
import pandas as pd
from tqdm import tqdm

In [ ]:
df = pd.read_csv('/content/candidates_posts.csv')

In [ ]:
# тут ключ от openai, на бесплатном скорее всего жесткий лимит
# на количество запросов в секунду, так что нужен платный
openai.api_key = "..."

In [ ]:
# системный промпт
# промпт можно доработать, так как gpt часто добавляет отсебятину, 
# если попросить отвечать в формате json, то должно быть лучше
# для экономии токенов можно перевести промпт на английский язык
address_extraction_template = 'Проверь указана ли в тексте проблема, связанная с работой ЖКХ, например, свалка, недостаточное освещение,'\
                              ' плохие дороги и т.п. Отвечай только одним словом: ДА или НЕТ. Если текст является рекламой чего-либо'\
                              ' или в тексте указан номер телефона, то отвечай НЕТ.'

In [ ]:
exmpl = """🌟Детский центр "Яркие дети" приглашает в летнюю школу "Формула успеха"!🌟Эта программа для вас, если вы остаётесь в городе и хотите, чтобы ваш ребенок:
👉Восстановил силы после учебы и провел увлекательно время без гаджетов
👉Расширил свой кругозор, подтянулся в чтении, освоил азы быстрого счета и приемы эффективного запоминания
👉Освоил навыки работы в команде, нашел новых друзей

Ребят ждёт:
1⃣Знакомство с яркими и увлекательными биографиями великих людей! Их восхождение к успеху, детство, интересы и увлечения помогут ребенку понять важность знаний, научат достигать высоких целей, преодолевать трудности, не сдаваться и верить в успех.

2⃣Интенсивные занятия по нескольким дисциплинам
Скорочтение, развитие памяти, коммуникативные тренинги, творческие мастер-классы, интеллектуальные игры, квесты, викторины, состязания и т.п.

3⃣Здоровое питание, прогулки на свежем воздухе, выездные мероприятия и экскурсии
Ежедневные 2-часовые прогулки, активные игры, футбольные матчи, выездные мероприятия (театры, музеи, выставки)

🧒👧🏼Возраст
7-12 лет
🥗Питание
3-х разовое - второй завтрак, полноценный обед, полдник
⏰Время пребывания
с 8:30 до 17:30
🏕Наши смены
29.05 - 18.06
19.06 - 09.07
10.07 - 30.07
31.07 - 20.08

Записывайтесь по телефону 📲 8(969) 210-61-66
📍Наш адрес: Петровский бульвар 5, вход с торца здания.
📍Наш сайт: https://spb.yarkiedeti.ru
📍Наша группа в ВК: https://vk.com/yarkie_deti_mur
⠀
#яркиедети #яркиедети_СПб #Муриноразвитие #летнийлагерь #каникулы #летодетям #летниеканикулы #развивающийцентр #детскийцентр #лепка #изо #лего #английскийязык #выходной #дляребенка #эмоциональныйинтеллект #литературныйклуб #Девяткинодетям"""

In [ ]:
# защита от пустых строк
def safe_str(s: str or None or float):
    return s if type(s) == str else ''

In [ ]:
# защита от ratelimit, когда сервис возвращает ошибку RateLimitError, то это значит,
# что случился ratelimit, надо подождать какое-то время, декоратор @backoff.on_exception
# позволяет на каждую ошибку RateLimitError автоматически ждать n секунд, затем пробовать
# сделать вызов опять, при этом время паузы увеличивается по экспоненте
@backoff.on_exception(backoff.expo, RateLimitError)
def completions_with_backoff(**kwargs):
    response = openai.ChatCompletion.create(**kwargs)
    return response

In [ ]:
# функция для использования gpt
# если в тексте больше max_text_threshold символов, он обрезается до нужной длины
# если текст слишком маленький, то в нем просто не может быть описана нормальная
# какая-либо ЖКХ проблема

short_text_threshold = 10
too_long_threshold = 1000
batch_size = 500


def task(text):
    text = safe_str(text)
    if len(text) < short_text_threshold or len(text) > too_long_threshold:
        return 'НЕТ.'
    messages = [
        {"role": "system", "content": address_extraction_template},
        {"role": "user", "content": text},
    ]
    response = completions_with_backoff(model="gpt-3.5-turbo", messages=messages, temperature=0, max_tokens=100)
    return response['choices'][0]['message']['content']

texts = df.text

# делается в несколько потоков для скорости, но если выставить слишком много потоков,
# то будет постоянно вылезать ошибка RateLimitError
for batch_begin in tqdm(range(0, len(texts), batch_size)):
    batch_end = min(batch_begin + batch_size, len(texts))

    # results = []
    results = [None] * (batch_end - batch_begin)

    # for i in range(batch_begin, batch_end):
    #     results.append(task(all_posts[i]['text']))

    with concurrent.futures.ThreadPoolExecutor(max_workers=2) as executor:
        future_to_response = {}
        for idx, post in enumerate(texts[batch_begin:batch_end]):
            future_to_response[executor.submit(task, post)] = idx
    for future in concurrent.futures.as_completed(future_to_response):
        idx = future_to_response[future]
        results[idx] = future.result()

    with open(f'/content/drive/MyDrive/вкр/chatgpt_candidates/batch_{batch_begin}_{batch_end - 1}.json', 'w', encoding='utf-8') as f:
        json.dump(results, f)

100%|██████████| 6/6 [19:32<00:00, 195.45s/it]


In [ ]:
results_dir = '/content/drive/MyDrive/вкр/chatgpt_candidates'

filenames = os.listdir(results_dir)
filenames.sort(key=lambda x: int(x.split('_')[1]))

answers = []
for filename in filenames:
    with open(os.path.join(results_dir, filename), 'r') as f:
        data = json.load(f)
    answers.extend(data)

In [ ]:
labels = list(map(lambda x: int(x.startswith('ДА') or x.startswith('Да')), answers))
# for i in range(len(all_posts)):
#     all_posts[i]['jkh_label'] = labels[i]
df['label'] = labels

In [ ]:
# with open('/content/drive/MyDrive/вкр/chatgpt_labelled.json', 'w', encoding='utf-8') as f:
#     json.dump(all_posts, f)
df.to_csv('/content/drive/MyDrive/вкр/candidates_posts_labelled.csv', sep='|', index=None)